In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# import tensorflow as tf
# if tf.test.gpu_device_name():
#     print('GPU found')
# else:
#     print("No GPU found")
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [2]:

def colab_download(proj_name,is_kaggle):
    import requests
    from tqdm import tqdm

    if is_kaggle:
        import sys
        sys.path.append("/kaggle/working/")

    project_name=proj_name

    url = f'http://kuroweb.tk:8000/api/v1/pack/wyftp/dataset/{project_name}.tar.gz'
    user, password = 'wuying', '123aaa'
    response = requests.get(url, stream=True,auth=(user, password))
    total_size_in_bytes= int(response.headers.get('content-length', 0))
    block_size = 1024 #1 Kibibyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
    with open('download.tar.gz', 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR, something went wrong")

    import tarfile

    # open file
    file = tarfile.open('download.tar.gz')
    # extracting file
    file.extractall('./')
    file.close()

    import shutil
    import os

    source_dir = f'./{project_name}'
    target_dir = './'

    file_names = os.listdir(source_dir)

    for file_name in file_names:
        try:
            shutil.move(os.path.join(source_dir, file_name), target_dir)
        except Exception as e:
            print(e,'at',file_name)




In [3]:

colab_download('model',True)
!pip install -r requirements.txt


In [4]:

import json

import numpy as np

# record start time
import pandas as pd
import torch
from IPython.core.display import display
from keras import layers
# residual block
# https://stackoverflow.com/questions/64792460/how-to-code-a-residual-block-using-two-layers-of-a-basic-cnn-algorithm-built-wit
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from general_model.grow_numpy import GrowableNumpyArray
from general_model.list_batch_loader import ListBatchLoadDataset
from model_helper.single_gpu_dataloader import SingleGpuTensorWrapper
from pytorch_regression.regression_params import RegressionParams
from tool import ml, wine_tool, data_loader, wine_data_loader
from tool.dbg import dbg

In [5]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [6]:
wine_json = ml.read_json("Wine.json")


class RegressionTrainer:
    def __init__(self):
        self.param = RegressionParams()
        self.param.batch_size = 64
        self.param.epoch = 1
        self.param.lr = 1e-9
        self.param.cnn_stride = 1
        self.param.cnn_size = 3
        self.param.pool_size = 2
        self.param.lr_decay = 0.99

    def load(self):

        self.x = GrowableNumpyArray(dtype=np.float, grow_speed=16)
        self.y = GrowableNumpyArray(dtype=np.float, grow_speed=16)

        for wine in wine_json:
            base_wine_path = 'data4/' + wine['id']
            dbg(wine['name'])
            if not os.path.exists(base_wine_path):
                dbg('not exists:', base_wine_path)
                continue
            # load data
            dirs = ml.getAllFiles(f'{base_wine_path}')
            for d in dirs:
                path = d
                files = ml.getAllFileRecursively(path)
                for f in files:
                    self.x.update(wine_tool.col2data_to_float_list(f))
                    self.y.add(wine['degree'])
                    # self.x = np.append(self.x, wine_tool.col2data_to_float_list(f))
                    # self.y = np.append(self.y, wine['degree'])

            # load item_data
            for wine_item in wine['items']:
                sub_wine_path = base_wine_path + '_' + wine_item['name']

                dirs = ml.getAllFiles(f'{sub_wine_path}')
                dbg(wine_item)
                for d in dirs:
                    path = d
                    files = ml.getAllFileRecursively(path)
                    for f in files[:100]:
                        self.x.update(wine_tool.col2data_to_float_list(f))
                        self.y.add(wine_item['degree'])
                        # self.x = np.append(self.x, wine_tool.col2data_to_float_list(f))
                        # self.y = np.append(self.y, wine_item['degree'])

        self.x = self.x.finalize()
        self.x = self.x.reshape((-1, 1000))

        self.y = self.y.finalize()

        plt.plot(sorted(self.y))
        plt.plot(self.y)

        # normalize

        self.y = self.y / 100

        layer_normed_x = torch.nn.LayerNorm([1000])(torch.tensor(self.x, dtype=torch.float))
        i = 0
        while i < 3:
            indice = i
            plt.plot(layer_normed_x[indice].detach().numpy(), label=self.y[indice])
            plt.legend()
            plt.show()
            i += 1

        self.x, self.y, self.x1, self.y1 = wine_data_loader.split(self.x, self.y, 1 - self.param.train_test_split)

        # wine_dataset = ListBatchLoadDataset(self.x.tensor, self.y.tensor)
        # self.dataloader = DataLoader(wine_dataset, batch_size=self.param.batch_size,
        #                         shuffle=True, num_workers=0)


trainer = RegressionTrainer()
trainer.load()
dbg(trainer.x.shape, trainer.y.shape)

In [7]:
tf.debugging.set_log_device_placement(False)

多层Dense模型

In [8]:

# act = 'tanh'
# # inp = __next = layers.Input(shape=[1280 * 720*10], name='input')
# inp = __next = layers.Input(shape=[1000], name='input')
# __next = layers.Dense(4000, activation=act)(__next)
# # __next = layers.Dense(800, activation=act)(__next)
# # __next = layers.Dense(800, activation=act)(__next)
# __next = layers.Dense(100, activation=act)(__next)
# __next = layers.Dense(10, activation=act)(__next)
# output = __next = layers.Dense(1, )(__next)
# model = keras.Model(inputs=[inp], outputs=[output])
# model.summary()
# # opt = tf.keras.optimizers.Adam(lr=1e-4)
# opt = tf.keras.optimizers.Adam(lr=1e-6)
# model.compile(
#     optimizer=opt,
#     # optimizer='adam',
#     loss='mse',
# )

In [9]:

kernel_count=3
act = 'tanh'
# inp = __next = layers.Input(shape=[1280 * 720*10], name='input')
inp = __next = layers.Input(shape=[1000], name='input')
__next = layers.Reshape(target_shape=[1000, 1], name='reshape')(__next)

for i in range(3):
    res=__next
    v1 = layers.Conv1D(filters=kernel_count, kernel_size=3,padding='same',activation=act)(res)
    v2 = layers.Conv1D(filters=kernel_count, kernel_size=5,padding='same',activation=act)(res)
    v3 = layers.Conv1D(filters=kernel_count, kernel_size=7,padding='same',activation=act)(res)
    v4 = layers.Conv1D(filters=kernel_count, kernel_size=9,padding='same',activation=act)(res)
    v5 = layers.Conv1D(filters=kernel_count, kernel_size=1,padding='same',activation=act)(res)
    
    __next=layers.Concatenate(axis=2)([v1,v2,v3,v4,v5])
    if i!=0:
        __next=res+__next
__next=layers.AvgPool1D(pool_size=50,strides=50)(__next)
__next = layers.Flatten()(__next)
__next = layers.Dense(10, activation=act)(__next)
output = __next = layers.Dense(1, )(__next)
model = keras.Model(inputs=[inp], outputs=[output])
model.summary()
# opt = tf.keras.optimizers.Adam(lr=1e-4)
opt = tf.keras.optimizers.Adam(lr=1e-6)
model.compile(
    optimizer=opt,
    # optimizer='adam',
    loss='mse',
)


In [10]:
# # loss:0.0147
# act = 'tanh'
# config = [
#     ('conv', 4, 3),
#     ('pool', 2),
#     ('activation',act),
#     ('conv', 4, 3),
#     ('pool', 2),
#     ('activation',act),

#     ('conv', 16, 3),
#     ('pool', 2),
#     ('activation',act),

#     ("conv", 16, 3),
#     ('pool', 2),
#     ('activation',act),

#     ("conv", 64, 3),
#     ('pool', 2),
#     ('activation',act),

#     ("conv", 64, 3),
#     ('pool', 2),
    
    
#     ("conv", 256, 3),
#     ('pool', 2),
#     ('activation',act),

#     ("conv", 1024, 3),
#     ('pool', 2),
#     ('activation',act),
# ]
# # inp = __next = layers.Input(shape=[1280 * 720*10], name='input')
# inp = __next = layers.Input(shape=[1000], name='input')
# __next = layers.Reshape(target_shape=[1000, 1], name='reshape')(__next)

# for c in config:
#     if c[0] == 'conv':
#         __next = layers.Conv1D(filters=c[1], kernel_size=c[2])(__next)
#     elif c[0] == 'pool':
#         __next = layers.MaxPool1D(pool_size=c[1])(__next)
#     elif c[0] == 'activation':
#         __next = layers.Activation(c[1])(__next)

# __next = layers.Flatten()(__next)
# output = __next = layers.Dense(1, )(__next)
# model = keras.Model(inputs=[inp], outputs=[output])
# model.summary()
# # opt = tf.keras.optimizers.Adam(lr=1e-4)
# opt = tf.keras.optimizers.Adam(lr=1e-4)
# model.compile(
#     optimizer=opt,
#     # optimizer='adam',
#     loss='mse',
# )

In [11]:
class CustomCallback(keras.callbacks.Callback):
    def __init__(self):
        self.val_fit=[]
        
    def on_epoch_begin(self, epoch, logs=None):
#         keys = list(logs.keys())
#         print("Start epoch {} of training; got log keys: {}".format(epoch, keys))
        pass
    def on_epoch_end(self, epoch, logs=None):
#         keys = list(logs.keys())
#         print("End epoch {} of training; got log keys: {}".format(epoch, keys))
        
        prediction = self.model.predict(trainer.x1)
        dif = prediction.reshape(-1, ) - trainer.y1
        percent=np.sum((np.abs(dif) < 0.025)) / len(dif) * 100
        self.val_fit.append(percent)

In [12]:
callback_for_percent=CustomCallback()

In [13]:
history = model.fit(trainer.x, trainer.y,
                    # validation_data=(X_test, Y_test)
#                     epochs=50,
#                     epochs=100,
#                     epochs=10000,
#                     epochs=400,
                    epochs=18000,
#                     epochs=15000,
#                     epochs=500,
                    # epochs=300,
                    callbacks=[callback_for_percent]
                    )

In [ ]:

pd.DataFrame(callback_for_percent.val_fit).plot(figsize=(8, 5))


In [ ]:
start_indice=(len(history.history["loss"])//3)
pd.DataFrame(history.history['loss'][start_indice:]).plot(figsize=(8, 5))
# plt.grid(True)
# plt.gca().set_ylim(0, 5e-4)
# plt.gca().set_ylim(0, 0.02)

plt.show()


prediction = model.predict(trainer.x1)
dict = {'predict': prediction[:15, 0],
        'truth': trainer.y1[:15],
        'diff': (prediction[:15, 0] - trainer.y1[:15])
        }
df = pd.DataFrame(dict)

# displaying the DataFrame
display(df.T)



In [ ]:
# 输出误差小于2.5度的百分比
dif = prediction.reshape(-1, ) - trainer.y1
np.sum((np.abs(dif) < 0.025)) / len(dif) * 100

# 调参记录

学习率调高到10的-4次方

训练轮次从500调整的5000，训练了3.2小时

换成mlp Dense模型

ver 9 10000 epoch只训练了不到1小时

ver 11 9w epoch，结果88%，还行，loss: 4.9452e-08，太低了 300-400 epoch就够了